<a href="https://colab.research.google.com/github/flexinJ/flexinBot/blob/main/flexinBollingerBandsW/Hurst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Downloading required packages
!pip install yfinance
!pip install numpy
!pip install pandas
!pip install hurst

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 322 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import of the necessary modules
import numpy as np
import pandas as pd
import math
import yfinance as yf
from hurst import compute_Hc
import matplotlib.pyplot as plt
from IPython.display import clear_output 
from time import sleep

In [ ]:
######################## Welcome ##########################
# Based in Bollinger bands, Hurt
# Analyze in M15 w/ input in M1
# Flexin (@moraisjota) make this ~ enjoy xD
######################## FlexinBot ########################

In [ ]:
class tradingSystem():
  def flexin(wallet, dateRange, timeFrame):
    for symbol in wallet:
      ######################## DataFrame ########################
      # Finance database import
      data = yf.Ticker(symbol).history(period=dateRange, interval=timeFrame)
      data.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
      # Calculate series of Log return of close prices 
      data['logCloseReturn'] = np.log(data['Close']/data['Close'].shift(1))
      ######################## Indicators ########################
      # Bollinger bands
      n = 20 # Moving average periods
      k = 2 # Factor to shift the bands
      data['Standard Deviation'] = data['Close'].rolling(n).std()
      data['Middle Band'] = data['Close'].rolling(n).mean()
      data['Upper Band'] = data['Middle Band'] + data['Standard Deviation']*k
      data['Lower Band'] = data['Middle Band'] - data['Standard Deviation']*k
      # Hurst exponent filter (https://github.com/Mottl/hurst)
      H = lambda x: compute_Hc(x)[0]
      data['Hurst'] = data['logCloseReturn'].rolling(window=100).apply(H)
      # Dealing with NaN data
      data.dropna(inplace=True, axis=0)
      ###################### Current values ######################
      # Price
      closePrice = data['Close'][-1] 
      openPrice = data['Open'][-1]
      # Bollinger bands
      middleBand = data['Middle Band'][-1] 
      upperBand = data['Upper Band'][-1]
      lowerBand = data['Lower Band'][-1]
      # Hurst exponent
      hurst = data['Hurst'][-1]
      print('-----------------------------------------------------------------')
      print('####################### Flexin strategy 🤖 ######################')
      ###################### Flexin strategy ######################
      if 1 > hurst > 0.5:
        print(f'>>> for {symbol}')
        print(f'WOW 🏂! H© = {hurst.round(2)} w/ persistent behavior.')
        if closePrice > upperBand: # Buy signal
          buy = True
          print(f'Price closes above the upper band. ⬆️')
          if True in buy:
            print(f'Uptrend probability. 📈📈📈')
        else:
          if closePrice < lowerBand: # Sell signal
            sell = True
            print(f'Price closes below the lower band. ⬇️')
            if True in sell:
              print(f'Downtrend probability. 📉📉📉')
      else:
        print(f'>>> for {symbol} -> H© = {hurst.round(2)} 💤')
        print('Wait...✋! Anti-persistent behavior.')

In [ ]:
while True:
  tradingSystem.flexin(['EURUSD=X', '^HSI'], '5d', '15m')